In [1]:
import itertools
import math
from typing import List, Optional, Sequence, Union

import numpy as np

In [2]:
basehash = hash


class Tiles:
    def __init__(
        self,
        dims_min: np.ndarray,
        dims_max: np.ndarray,
        tiling_dim: int,
        num_tilings: Optional[int] = None,
    ):
        assert isinstance(dims_min, np.ndarray)
        assert isinstance(dims_max, np.ndarray)
        self.dims_max = dims_max
        self.dims_min = dims_min
        self.tiling_dim = tiling_dim
        self.wrapwidths = [tiling_dim] * np.size(dims_min)

        # num tilings should a power of 2
        # and at least 4 times greater than
        # the number of dimensions
        self.num_tilings = num_tilings or pow2geq(np.size(dims_min) * 4)
        self.max_size = (tiling_dim ** np.size(dims_min)) * self.num_tilings
        print("Num tilings", self.num_tilings, "\n", "Flat dim:", self.max_size)
        self.iht = IHT(self.max_size)

    def __call__(self, xs):
        xs_scaled_01 = (xs - self.dims_min) / (self.dims_max - self.dims_min)
        repr_ = np.zeros(shape=self.max_size)
        idx = tileswrap(
            self.iht, self.num_tilings, xs_scaled_01 * self.tiling_dim, self.wrapwidths
        )
        repr_[idx] = 1
        return repr_


class IHT:
    """
    Structure to handle collisions
    Source: http://incompleteideas.net/tiles/tiles3.html

    """

    def __init__(self, sizeval):
        self.size = sizeval
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        "Prepares a string for printing whenever this object is printed"
        return (
            "Collision table:"
            + " size:"
            + str(self.size)
            + " overfullCount:"
            + str(self.overfullCount)
            + " dictionary:"
            + str(len(self.dictionary))
            + " items"
        )

    def count(self):
        return len(self.dictionary)

    def fullp(self):
        return len(self.dictionary) >= self.size

    def getindex(self, obj, readonly=False):
        d = self.dictionary
        if obj in d:
            return d[obj]
        elif readonly:
            return None
        size = self.size
        count = self.count()
        if count >= size:
            # TODO: Fail
            if self.overfullCount == 0:
                print("IHT full, starting to allow collisions")
            self.overfullCount += 1
            return basehash(obj) % self.size
        else:
            d[obj] = count
            return count


def hashcoords(coordinates, m, readonly=False):
    if isinstance(m, IHT):
        return m.getindex(tuple(coordinates), readonly)
    if isinstance(m, int):
        return basehash(tuple(coordinates)) % m
    if m is None:
        return coordinates


def tiles(
    ihtORsize: Union[IHT, int, None],
    numtilings: int,
    floats: List[float],
    ints: List[int] = [],
    readonly: bool = False,
) -> List[int]:
    """returns num-tilings tile indices corresponding to the floats and ints"""
    qfloats = [math.floor(f * numtilings) for f in floats]
    tiles_ = []
    for tiling in range(numtilings):
        tiling_x2 = tiling * 2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append((q + b) // numtilings)
            b += tiling_x2
        coords.extend(ints)
        tiles_.append(hashcoords(coords, ihtORsize, readonly))
    return tiles_


def tileswrap(
    ihtORsize: Union[IHT, int, None],
    numtilings: int,
    floats: Sequence[float],
    wrapwidths: Sequence[int],
    ints: Sequence[int] = [],
    readonly: bool = False,
) -> Sequence[int]:
    """returns num-tilings tile indices corresponding to the floats and ints, wrapping some floats"""
    qfloats = [math.floor(f * numtilings) for f in floats]
    tiles_ = []
    for tiling in range(numtilings):
        tiling_x2 = tiling * 2
        coords = [tiling]
        b = tiling
        for q, width in itertools.zip_longest(qfloats, wrapwidths):
            c = (q + b % numtilings) // numtilings
            coords.append(c % width if width else c)
            b += tiling_x2
        coords.extend(ints)
        tiles_.append(hashcoords(coords, ihtORsize, readonly))
    return tiles_


def pow2geq(lb: int) -> int:
    exp = 1
    rs = 1
    while True:
        rs = 2**exp
        if rs >= lb:
            break
        exp += 1
    return rs


In [3]:
def bits_required(n: int) -> int:
    """
    Returns the number of bits required to represent an integer in binary
    """
    if n == 0:
        return 1
    return len(bin(abs(n))[2:]) 


def int_to_binary_array(n: int, width: Optional[int] = None) -> np.ndarray:
    if width is None:
        width = bits_required(n)
        
    binary = np.zeros(width, dtype=np.int8)
    n = abs(n)  # Handle negative numbers
    
    # Get binary digits
    digits = []
    while n:
        digits.append(n & 1)
        n >>= 1
        
    # Pad with zeros if needed
    digits.extend([0] * (width - len(digits)))
    
    # Take only width digits and reverse to get proper order
    binary[:] = digits[:width][::-1]
    
    return binary

print(int_to_binary_array(5, width=5))
print(int_to_binary_array(5))

[0 0 1 0 1]
[1 0 1]


In [4]:
def interger_to_sequence(
    space_size: int, sequence_length: int, index: int
) -> Sequence[int]:
    """
    Uses the positional system of integers to generate a unique
    sequence of numbers given represetation integer - `index`.

    Based on https://2ality.com/2013/03/permutations.html.

    Args:
        space_size: the number of possible digits
        sequence_length: the length of the sequence of digits.
        index: the index of the unique sequence.
    """
    xs = []
    for pw in reversed(range(sequence_length)):
        if pw == 0:
            xs.append(index)
        else:
            mult = space_size**pw
            digit = math.floor(index / mult)
            xs.append(digit)
            index = index % mult
    return tuple(xs)

print(interger_to_sequence(2, 5, 5))

(0, 0, 1, 0, 1)


In [5]:
size = 0
delay = 10
for i in range(2, 10):
    n = 8 ** i
    size += n
    print(i, n)


bits_required(size)

2 64
3 512
4 4096
5 32768
6 262144
7 2097152
8 16777216
9 134217728


28

In [6]:
min_delay, max_delay = 2, 6 # for options
num_actions = 8
actions = tuple(range(num_actions))

In [7]:
options_mapping = {
    length:  len(actions) ** length for length in range(min_delay, max_delay)
}
options_mapping

{2: 64, 3: 512, 4: 4096, 5: 32768}

In [8]:
num_options = sum(value for value in options_mapping.values())
num_options

37440

In [9]:
tiling_dim = 2
obs_dim = 1 + max_delay
num_tilings = pow2geq(obs_dim * 4)
print(num_tilings)
max_size = (
    (tiling_dim ** obs_dim) * num_tilings
)
max_size

32


4096

In [10]:
xs = np.array([1, 2, 3])
ys = np.array([5, 6, 7])
np.concatenate([xs[:,None], ys[:,None]], axis=1)

array([[1, 5],
       [2, 6],
       [3, 7]])

In [11]:
2**17


131072

In [12]:
b = np.random.rand(150000, 400)
print(b.nbytes / 1024 / 1024)

457.763671875


In [13]:
a = np.array([0, 1, 2])

np.tile(a, (100,1))

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2],
       [0,